# Download Data Section

In [1]:
from bs4 import BeautifulSoup
import requests

url = 'https://dd.weather.gc.ca/alerts/cap/'
ext = '*'

def listFD(url, ext=''):
    page = requests.get(url).text

    soup = BeautifulSoup(page, 'html.parser')
    return [url + '/' + node.get('href') for node in soup.find_all('a') ]#if node.get('href').endswith(ext)]

dic_list = listFD(url, ext)



In [ ]:
for file in dic_list:
    print(file.split('/')[-2])





alerts
20180514
20180515
20180516
20180517
20180518
20180519
20180520
20180521
20180522
20180523
20180524
20180525
20180526
20180527
20180528
20180529
20180530
20180531
20180601
20180602
20180603
20180604
20180605
20180606
20180607
20180608
20180609
20180610
20180611
20180612
20180613
20180614
20180615
20180616
20180617
20180618
20180619
20180620
20180621
20180622
20180623
20180624
20180625
20180626
20180627
20180628
20180629
20180630
20180701
20180702
20180703
20180704
20180705
20180706
20180707
20180708
20180709
20180809
20180810
20180811
20180812
20180813
20180814
20180815
20180816
20180817
20180818
20180819
20180820
20180821
20180822
20180823
20180824
20180825
20180826
20180827
20180828
20180829
20180830
20180831
20180901
20180902
20180903
20180904
20180905
20180906
20180907
20180908
20180909
20180910
20180911
20180912
20180913
20180914
20180915
20180916
20180917
20180918
20180919
20180920
20180921
20180922
20180923
20180924
20180925
20180926
20180927
20180928
20180929
20180930

NameError: name 'nfiodaf' is not defined

# Sound Generation

In [11]:
from pyo import *
import random

s = Server().boot()

# Sound bank
folder = "snds/"
sounds = ["alum1.wav", "alum2.wav", "alum3.wav", "alum4.wav"]

# Creates the left and right players
sfL = SfPlayer(folder+sounds[0], speed=1, mul=0.5).out()
sfR = SfPlayer(folder+sounds[0], speed=1, mul=0.5).out(1)

# Function to choose a new sound and a new speed for the left player
def newL():
    sfL.path = folder + sounds[random.randint(0, 3)]
    sfL.speed = random.uniform(0.75, 1.5)
    sfL.out()                                                                       

# The "end-of-file" signal triggers the function "newL"
tfL = TrigFunc(sfL["trig"], newL)

# Function to choose a new sound and a new speed for the right player
def newR():
    sfR.path = folder + sounds[random.randint(0, 3)]
    sfR.speed = random.uniform(0.75, 1.5)
    sfR.out(1)

# The "end-of-file" signal triggers the function "newR"
tfR = TrigFunc(sfR["trig"], newR)

In [12]:
s.stop()


In [ ]:
from pyo import *
import random

s = Server(duplex=0).boot()

# Compute the duration, in seconds, of one buffer size.
buftime = s.getBufferSize() / s.getSamplingRate()

# Delay parameters
delay_time_l = Sig(0.125)   # Delay time for the left channel delay.
delay_time_l.ctrl()
delay_feed = Sig(0.75)      # Feedback value for both delays.
delay_feed.ctrl()

# Because the right delay gets its input sound from the left delay, while
# it is computed before (to send its output sound to the left delay), it
# will be one buffer size late. To compensate this additional delay on the
# right, we substract one buffer size from the real delay time.
delay_time_r = Sig(delay_time_l, add=-buftime)

# Setup up a soundfile player.
sf = SfPlayer('snds/alum1.wav').stop()

# Send the original sound to both speakers.
sfout = sf.mix(2).out()

# Initialize the right delay with zeros as input because the left delay
# does not exist yet.
right = Delay(Sig(0), delay=delay_time_r).out(1)

# Initialize the left delay with the original mono source and the right
# delay signal (multiplied by the feedback value) as input.
left = Delay(sf + right * delay_feed, delay=delay_time_l).out()

# One issue with recursive cross-delay is if we set the feedback to
# 0, the right delay never gets any signal. To resolve this, we add a
# non-recursive delay, with a gain that is the inverse of the feedback,
# to the right delay input.
original_delayed = Delay(sf, delay_time_l, mul=1-delay_feed)

# Change the right delay input (now that the left delay exists).
right.setInput(original_delayed + left * delay_feed)

def playit():
    "Assign a sound to the player and start playback."
    which = random.randint(1, 4)
    path = "snds/alum%d.wav" % which
    sf.path = path
    sf.play()

# Call the function "playit" every second.
pat = Pattern(playit, 1).play()
s.gui(locals())

In [ ]:
s.top()

In [ ]:
from pyo import *

# Creates and boots the server.
# The user should send the "start" command from the GUI.
s = Server().boot()
# Drops the gain by 20 dB.
s.amp = 0.1

# Creates a sine wave player.
# The out() method starts the processing
# and sends the signal to the output.
a = Sine().out()
s.gui(locals())

In [2]:
s.stop()